# Links

- [ ] https://cran.r-project.org/web/packages/afex/vignettes/introduction-mixed-models.pdf
- [ ] https://cran.r-project.org/web/packages/afex/afex.pdf
- [ ] https://cran.r-project.org/web/packages/afex/index.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_analysing_accuracy_data.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_anova_example.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/messy-data.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/interactions.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/basics.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html


# Setup

## Imports

In [2]:
library("afex")     # needed for mixed() and attaches lme4 automatically.
library("emmeans")  # emmeans is needed for follow-up tests
library("multcomp") # for advanced control for multiple testing/Type 1 errors.
library("ggplot2")  # for plots

In [ ]:
library("car")
library("dplyr")
library("tidyr")
require("lattice")
require("ggplot2")
require("ez")

## Settings

In [3]:
theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))

In [4]:
options(width=120)

## Handy functions

### Logging information BEFORE execution

In [23]:
logging0 <- function(ifName,infoStr=""){
    t0 <- Sys.time()
    s0 <- gsub("\\s","_",t0)
    s1 <- gsub("^.","",tempfile("","",""))
    s2 <- gsub("/t1006/","/t1007/",ifName)
    ofName <- paste(paste(s2,"",s0,"",s1,infoStr,"_nrow",nrow(df3),sep="_"),".RData",sep="")
    lfName = paste(ofName ,".LOG.txt" ,sep="" ,collapse = "")
    log_msg <- toString(paste(
        paste("logs0" ,toString(lfName)     ,sep=": " ,collapse = "")
      , paste("time0" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("nrows" ,toString(nrow(df3))  ,sep=": " ,collapse = "")
      , paste("ncols" ,toString(ncol(df3))  ,sep=": " ,collapse = "")
      , paste("vList" ,toString(ls())       ,sep=": " ,collapse = "")
      , paste("oSave" ,toString(ofName)     ,sep=": " ,collapse = "")        
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("ofName"=ofName,"lfName"=lfName,"t0"=t0))
}

### Logging information AFTER execution

In [25]:
logging1 <- function(lfName,t0){
    t1 <- Sys.time()
    d1 <- t1-t0
    log_msg <- toString(paste(
        paste("time1" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("diff1" ,capture.output(d1)   ,sep=": " ,collapse = "")
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("d1" = d1))
}

# Data Load

Select one of the files (by moving lines around) depending on how much data you want to include in the execution (terminal line containing `ifName` defines the file to be loaded).

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e3x3p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e3x6p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e3x8p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e2x1p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e3x1p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e3x2p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e3x4p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_fullset_e6x1p5.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e2x5p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_subsamp_e4x1p0.RData"

In [ ]:
ifName="data/t1006/bigDF__df3_fullset_e6x1p5.RData"

In [37]:
load(file=ifName)

In [38]:
str(df3)

'data.frame':	1505108 obs. of  5 variables:
 $ CAMP      : Factor w/ 2 levels "metoo","sexstrike": 1 1 1 1 1 1 1 1 1 1 ...
 $ TYPE      : Factor w/ 2 levels "ctrl","orig": 2 2 2 2 2 2 2 2 2 2 ...
 $ ID        : chr  "919365304114929664" "919365304114929664" "919366579497734144" "919366579497734144" ...
 $ MEAS_LEVEL: Factor w/ 2 levels "vb01Prc","ag01Prc": 1 2 1 2 1 2 1 2 1 2 ...
 $ MEAS_VALUE: num  0 0 0 0 0.0526 ...


In [39]:
summary(df3)

        CAMP           TYPE             ID              MEAS_LEVEL       MEAS_VALUE     
 metoo    :1352076   ctrl:827952   Length:1505108     vb01Prc:752554   Min.   :0.00000  
 sexstrike: 153032   orig:677156   Class :character   ag01Prc:752554   1st Qu.:0.00000  
                                   Mode  :character                    Median :0.00000  
                                                                       Mean   :0.04155  
                                                                       3rd Qu.:0.06667  
                                                                       Max.   :1.00000  

# Model definition

| variable/factor type        | variable name | symbol |
|:----------------------------|:--------------|--------|
| observation unit (tweet_id) | ID            | u1     |
| outcome (dependent)         | MEAS_VALUE    | d1     |
| between tweet               | CAMP          | b1     |
| between tweet               | TYPE          | b2     |
| within tweet                | MEAS_LEVEL    | w1     |

In [ ]:
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(MEAS_LEVEL||ID)

In [40]:
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(1|ID)

In [ ]:
CONTROL <- lmerControl(optCtrl=list(maxfun=1e7))

In [41]:
CONTROL <- lmerControl(optCtrl=list(maxfun=1e6))

In [ ]:
METHOD <- "LRT" # calculates p-vals via the likelihood ratio tests (use if random factors have > 50 levels)

In [ ]:
METHOD <- "KR"  # Kenward-Roger approximation for degrees of freedom

In [52]:
METHOD <- "S"   # Satterthwaite approximation for degrees of freedom

In [43]:
log0 <- logging0(ifName,infoStr="_AFEX_MIXED_formula0")
m0.0 <- mixed(
    formula   = FORMULA,
    data      = df3,
    control   = CONTROL,
    method    = METHOD,
    expand_re = TRUE
)
save(m0.0,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

logs0: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_01:52:47__e953dd3b8f1__AFEX_MIXED_formula0__nrow_1505108.RData.LOG.txt
time0: 2020-09-30 01:52:47
nrows: 1505108
ncols: 5
vList: ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_01:52:47__e953dd3b8f1__AFEX_MIXED_formula0__nrow_1505108.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE, MEAS_LEVEL, ID



Fitting one lmer() model. [DONE]
Calculating p-values. [DONE]
time1: 2020-09-30 01:55:06
diff1: Time difference of 2.304855 mins



In [44]:
m0.0

Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (1 | ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.48 1231.90 ***   <.001
2                 TYPE 1, 752550.48   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.90 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.48  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.90   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.90        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.90   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

# Inspect

In [61]:
m0 <- m0.0
m0

Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (1 | ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.48 1231.90 ***   <.001
2                 TYPE 1, 752550.48   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.90 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.48  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.90   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.90        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.90   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

In [54]:
m0 <- m0.1
m0

Warning message:
“lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.00236551 (tol = 0.002, component 1)”


Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (MEAS_LEVEL || ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.58 1231.90 ***   <.001
2                 TYPE 1, 752550.58   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.55 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.58  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.55   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.55        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.55   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

In [62]:
summary(m0)$varcor

 Groups   Name        Std.Dev.
 ID       (Intercept) 0.035764
 Residual             0.059575

In [63]:
m3.0 <- emmeans(m0, c("CAMP", "TYPE", "MEAS_LEVEL"))
m3.0

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.



 CAMP      TYPE MEAS_LEVEL  emmean        SE  df asymp.LCL asymp.UCL
 metoo     ctrl vb01Prc    0.04612 0.0001143 Inf   0.04590   0.04635
 sexstrike ctrl vb01Prc    0.04977 0.0003296 Inf   0.04912   0.05041
 metoo     orig vb01Prc    0.04167 0.0001255 Inf   0.04142   0.04191
 sexstrike orig vb01Prc    0.05100 0.0003880 Inf   0.05024   0.05176
 metoo     ctrl ag01Prc    0.04085 0.0001143 Inf   0.04063   0.04108
 sexstrike ctrl ag01Prc    0.04409 0.0003296 Inf   0.04344   0.04474
 metoo     orig ag01Prc    0.03362 0.0001255 Inf   0.03338   0.03387
 sexstrike orig ag01Prc    0.04737 0.0003880 Inf   0.04661   0.04813

Degrees-of-freedom method: asymptotic 
Confidence level used: 0.95 

In [64]:
pairs(m3.0)

 contrast                                         estimate       SE  df z.ratio p.value
 metoo ctrl vb01Prc - sexstrike ctrl vb01Prc     -0.003645 0.000349 Inf -10.450 <.0001 
 metoo ctrl vb01Prc - metoo orig vb01Prc          0.004454 0.000170 Inf  26.237 <.0001 
 metoo ctrl vb01Prc - sexstrike orig vb01Prc     -0.004878 0.000405 Inf -12.058 <.0001 
 metoo ctrl vb01Prc - metoo ctrl ag01Prc          0.005269 0.000139 Inf  38.017 <.0001 
 metoo ctrl vb01Prc - sexstrike ctrl ag01Prc      0.002031 0.000349 Inf   5.823 <.0001 
 metoo ctrl vb01Prc - metoo orig ag01Prc          0.012497 0.000170 Inf  73.618 <.0001 
 metoo ctrl vb01Prc - sexstrike orig ag01Prc     -0.001248 0.000405 Inf  -3.084 0.0427 
 sexstrike ctrl vb01Prc - metoo orig vb01Prc      0.008099 0.000353 Inf  22.965 <.0001 
 sexstrike ctrl vb01Prc - sexstrike orig vb01Prc -0.001232 0.000509 Inf  -2.421 0.2310 
 sexstrike ctrl vb01Prc - metoo ctrl ag01Prc      0.008914 0.000349 Inf  25.554 <.0001 
 sexstrike ctrl vb01Prc - sexstr

In [65]:
summary(as.glht(pairs(m3.0)), test=adjusted("free"))

Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > 


	 Simultaneous Tests for General Linear Hypotheses

Linear Hypotheses:
                                                       Estimate Std. Error z value Pr(>|z|)    
metoo ctrl vb01Prc - sexstrike ctrl vb01Prc == 0     -0.0036452  0.0003488 -10.450  < 2e-16 ***
metoo ctrl vb01Prc - metoo orig vb01Prc == 0          0.0044540  0.0001698  26.237  < 2e-16 ***
metoo ctrl vb01Prc - sexstrike orig vb01Prc == 0     -0.0048777  0.0004045 -12.058  < 2e-16 ***
metoo ctrl vb01Prc - metoo ctrl ag01Prc == 0          0.0052691  0.0001386  38.017  < 2e-16 ***
metoo ctrl vb01Prc - sexstrike ctrl ag01Prc == 0      0.0020313  0.0003488   5.823 2.11e-08 ***
metoo ctrl vb01Prc - metoo orig ag01Prc == 0          0.0124974  0.0001698  73.618  < 2e-16 ***
metoo ctrl vb01Prc - sexstrike orig ag01Prc == 0     -0.0012476  0.0004045  -3.084  0.00407 ** 
sexstrike ctrl vb01Prc - metoo orig vb01Prc == 0      0.0080992  0.0003527  22.965  < 2e-16 ***
sexstrike ctrl vb01Prc - sexstrike orig vb01Prc == 0 -0.0012325 

In [66]:
m2.0 <- emmeans(m0,c("CAMP","TYPE"))
m2.0
pairs(m2.0)
summary(as.glht(pairs(m2.0)),test=adjusted("free"))


Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.

NOTE: Results may be misleading due to involvement in interactions



 CAMP      TYPE  emmean        SE  df asymp.LCL asymp.UCL
 metoo     ctrl 0.04349 9.091e-05 Inf   0.04331   0.04367
 sexstrike ctrl 0.04693 2.621e-04 Inf   0.04642   0.04744
 metoo     orig 0.03765 9.981e-05 Inf   0.03745   0.03784
 sexstrike orig 0.04918 3.086e-04 Inf   0.04858   0.04979

Results are averaged over the levels of: MEAS_LEVEL 
Degrees-of-freedom method: asymptotic 
Confidence level used: 0.95 

 contrast                        estimate       SE  df z.ratio p.value
 metoo ctrl - sexstrike ctrl     -0.00344 0.000277 Inf -12.405 <.0001 
 metoo ctrl - metoo orig          0.00584 0.000135 Inf  43.266 <.0001 
 metoo ctrl - sexstrike orig     -0.00570 0.000322 Inf -17.710 <.0001 
 sexstrike ctrl - metoo orig      0.00928 0.000280 Inf  33.097 <.0001 
 sexstrike ctrl - sexstrike orig -0.00226 0.000405 Inf  -5.571 <.0001 
 metoo orig - sexstrike orig     -0.01154 0.000324 Inf -35.576 <.0001 

Results are averaged over the levels of: MEAS_LEVEL 
Degrees-of-freedom method: asymptotic 
P value adjustment: tukey method for comparing a family of 4 estimates 


	 Simultaneous Tests for General Linear Hypotheses

Linear Hypotheses:
                                       Estimate Std. Error z value Pr(>|z|)    
metoo ctrl - sexstrike ctrl == 0     -0.0034415  0.0002774 -12.405  < 2e-16 ***
metoo ctrl - metoo orig == 0          0.0058412  0.0001350  43.266  < 2e-16 ***
metoo ctrl - sexstrike orig == 0     -0.0056972  0.0003217 -17.710  < 2e-16 ***
sexstrike ctrl - metoo orig == 0      0.0092827  0.0002805  33.097  < 2e-16 ***
sexstrike ctrl - sexstrike orig == 0 -0.0022557  0.0004049  -5.571 2.53e-08 ***
metoo orig - sexstrike orig == 0     -0.0115384  0.0003243 -35.576  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- free method)


# Experimantal stuff

## Formula 1

In [45]:
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(MEAS_LEVEL||ID)

In [46]:
log0 <- logging0(ifName,infoStr="_AFEX_MIXED_formula1")
m0.1 <- mixed(
    formula   = FORMULA,
    data      = df3,
    control   = CONTROL,
    method    = METHOD,
    expand_re = TRUE
)
save(m0.1,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

logs0: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_01:55:06__e9570bb90ec__AFEX_MIXED_formula1__nrow_1505108.RData.LOG.txt
time0: 2020-09-30 01:55:06
nrows: 1505108
ncols: 5
vList: ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_01:55:06__e9570bb90ec__AFEX_MIXED_formula1__nrow_1505108.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE, MEAS_LEVEL, ID



Fitting one lmer() model. 

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00236551 (tol = 0.002, component 1)”


[DONE]
Calculating p-values. [DONE]
time1: 2020-09-30 01:58:42
diff1: Time difference of 3.595648 mins



In [47]:
m0.1

Warning message:
“lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.00236551 (tol = 0.002, component 1)”


Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (MEAS_LEVEL || ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.58 1231.90 ***   <.001
2                 TYPE 1, 752550.58   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.55 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.58  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.55   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.55        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.55   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

# Helpers

In [ ]:
?mixed